In [32]:
import json
import psycopg2
from decimal import Decimal

In [33]:
# Connect to the PostgreSQL database
conn = psycopg2.connect("dbname=creditcompass user=postgres password=constella")
cur = conn.cursor()

In [34]:
# # Fetch data from PostgreSQL
# queries = {
#     "loansapproved": "SELECT addr_state, COUNT(*) as loansapproved FROM loans GROUP BY addr_state;",
#     "percent_loans_paid": "SELECT addr_state, (COUNT(CASE WHEN loan_status = 'Paid' THEN 1 END) * 100.0 / COUNT(*)) as percent_loans_paid FROM loans GROUP BY addr_state;",
#     "avg_income_to_loan_ratio": "SELECT addr_state, AVG(income_to_loan_ratio) as avg_income_to_loan_ratio FROM loans GROUP BY addr_state;",
#     "sum_loan_amnt": "SELECT addr_state, SUM(loan_amnt) as avg_loan_amnt FROM loans GROUP BY addr_state;"
# }

queries = {
    "loansapproved": """
        SELECT addr_state, COUNT(*) as loansapproved 
        FROM loans 
        GROUP BY addr_state;
    """,
    
    "percent_loans_paid": """
        SELECT addr_state, 
               ROUND(COUNT(CASE WHEN loan_status = 'Paid' THEN 1 END) * 100.0 / COUNT(*), 2) as percent_loans_paid 
        FROM loans 
        GROUP BY addr_state;
    """,
    
    "avg_income_to_loan_ratio": """
        SELECT addr_state, 
               ROUND(AVG(income_to_loan_ratio), 2) as avg_income_to_loan_ratio 
        FROM loans 
        GROUP BY addr_state;
    """,
    
    "sum_loan_amnt": """
        SELECT addr_state, 
               TO_CHAR(SUM(loan_amnt), '999,999,999.00') as sum_loan_amnt 
        FROM loans 
        GROUP BY addr_state;
    """
}



In [35]:
data = {}
for key, query in queries.items():
    cur.execute(query)
    rows = cur.fetchall()
    data[key] = {row[0]: row[1] for row in rows}

cur.close()
conn.close()

UndefinedFunction: function round(double precision, integer) does not exist
LINE 3:                ROUND(AVG(income_to_loan_ratio), 2) as avg_in...
                       ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.


In [ ]:
# Function to convert Decimal to float
def convert_decimals(obj):
    if isinstance(obj, dict):
        return {k: convert_decimals(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_decimals(i) for i in obj]
    elif isinstance(obj, Decimal):
        return float(obj)
    else:
        return obj


In [ ]:
# Load the existing JSON file
with open('static/us-states.json', 'r') as f:
    json_data = json.load(f)

In [ ]:
# Add data to JSON
for feature in json_data['features']:
    state_id = feature['id']
    properties = feature['properties']
    properties['loansapproved'] = data['loansapproved'].get(state_id, 0)
    properties['percent_loans_paid'] = data['percent_loans_paid'].get(state_id, 0.0)
    properties['avg_income_to_loan_ratio'] = data['avg_income_to_loan_ratio'].get(state_id, 0.0)
    properties['sum_loan_amnt'] = data['sum_loan_amnt'].get(state_id, 0.0)

In [ ]:
# Convert Decimal values before saving
json_data = convert_decimals(json_data)

In [ ]:
# Save the updated JSON file
with open('static/us-states-updated.json', 'w') as f:
    json.dump(json_data, f, indent=4)

print("Updated JSON file saved as 'static/us-states-updated.json'")


Updated JSON file saved as 'static/us-states-updated.json'
